In [1]:
from inspect import currentframe, getframeinfo
from time import time
from datetime import datetime
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model, save_model, Model
from keras import backend as K 
import os
import keras
import random
import numpy as np
import pprint

from common.config import *
from common.util import logm, parse_csv, Timer
from common.dataloader import (wav_to_specdata, normalize, spec_load_and_rshp,
                               spec_save)
from common.datasets import Dataset, TestDataset
from common.generator import Generator, Heap
from common.callbacks import TimedStopping
from evaluation import evaluate
from model import build_model
from params import (params, can_shake, create_space,
                    params_keys_comma_separated, params_values_comma_separated)

Using TensorFlow backend.


In [2]:
DEVELOPING = False
RUNS = 100
TIME_LIMIT = "25/09/2019 12:00:00"

In [3]:
def setup_dirs(conf: Config):
    logm('Setting up directories...', cur_frame=currentframe(),
         mtype='I')

    models_checkpoint_dir = os.path.join(MODELS_CHECKPOINT_DIR,
                                         str(conf))
    os.makedirs(models_checkpoint_dir, exist_ok=True)
    conf.models_checkpoint_dir = models_checkpoint_dir
    logm('Setting up directories: the model checkpoints directory is '
         f'{conf.models_checkpoint_dir}',
         cur_frame=currentframe(), mtype='I')
    log_dir = os.path.join(LOG_DIR, str(conf))
    os.makedirs(log_dir, exist_ok=True)
    conf.log_dir = log_dir
    logm(f'Setting up directories: the log dir is {conf.log_dir}',
         cur_frame=currentframe(), mtype='I')

    models_dir = os.path.join(MODELS_DIR, str(conf))
    os.makedirs(models_dir, exist_ok=True)
    conf.models_dir = models_dir
    logm('Setting up directories: the model directory is '
         f'{conf.models_dir}',
         cur_frame=currentframe(), mtype='I')

    heap_dir = HEAP_DIR
    os.makedirs(heap_dir, exist_ok=True)
    conf.heap_dir = heap_dir
    logm('Setting up directories: the heap directory is '
         f'{conf.heap_dir}',
         cur_frame=currentframe(), mtype='I')

    logm('Setting up directories... Done', cur_frame=currentframe(),
         mtype='I')

def setup_clbks(conf: Config,
                setup_tb=False,
                setup_mc=False,
                setup_ts=True,
                setup_es=False):
    logm('Setting up callbacks', cur_frame=currentframe(), mtype='I')
    callbacks = []
    if setup_tb:
        logm('Setting up tensorboard', cur_frame=currentframe(), mtype='I')
        if conf.use_tb_embeddings:
            paths, labels = parse_csv(conf.test_data_csv,
                                      conf.data_path)
            test_set = TestDataset(paths, labels)

            with open(os.path.join(conf.log_dir, 'metadata.tsv'), 'w') as f:
                np.savetxt(f, labels, delimiter=",", fmt='%s')

            logm(f'Loading test data ({len(paths)} samples) '
                 'for tensorboard callback...',
                 cur_frame=currentframe(), mtype='I')
            y_test = test_set()[1]
            x_test = np.asarray([conf.data_loader(x) for x in test_set()[0]])

            logm(f'Loading test data ({len(paths)} samples) for '
                 'tensorboard callback... Done',
                 cur_frame=currentframe(), mtype='I')

            print('x_test shape:', x_test.shape)
            tb = TensorBoard(
                log_dir=os.path.join(conf.log_dir, 'tensorboard'),
                histogram_freq=1,
                batch_size=conf.batch_size,
                write_graph=True,
                write_grads=True,
                write_images=True,
                embeddings_freq=5,
                embeddings_layer_names=['features'],
                embeddings_metadata='metadata.tsv',
                embeddings_data=x_test)
        else:
            tb = TensorBoard(log_dir=conf.log_dir,
                             histogram_freq=1,
                             batch_size=conf.batch_size,
                             write_graph=True)
        callbacks.append(tb)
    if setup_ts:
        logm('Setting up TimedStopping',
             cur_frame=currentframe(),
             mtype='I')
        if conf.max_seconds_per_run:
            callbacks.append(TimedStopping(conf.max_seconds_per_run))
        else:
            logm('Could not set up TimedStopping: '
                 'conf.max_seconds_per_run is set as None',
                 cur_frame=currentframe(),
                 mtype='W')
    if setup_es:
        logm('Setting up EarlyStopping',
             cur_frame=currentframe(),
             mtype='I')
        callbacks.append(EarlyStopping(patience=5))
    if setup_mc:
        logm('Setting up ModelCheckpoint',
             cur_frame=currentframe(),
             mtype='I')
        callbacks.append(ModelCheckpoint(
            f'{conf.model_checkpoint_location}.h5',
            period=1,
            save_best_only=True))

    logm('Setting up callbacks... Done',
         cur_frame=currentframe(),
         mtype='I')
    return callbacks


def setup_heap(conf: Config):
    logm('Setting up heap...', cur_frame=currentframe(), mtype='I')
    logm(f'Setting up heap: the heap directory is {conf.heap_dir}',
         cur_frame=currentframe(), mtype='I')
    if not conf.use_heap:
        logm('Setting up heap: heap is deactivated',
             cur_frame=currentframe(), mtype='W')
        return None
    heap = Heap(load_fn=spec_load_and_rshp,
                save_fn=spec_save,
                file_format='png',
                heap_dir=HEAP_DIR)
    logm('Setting up heap... Done', cur_frame=currentframe(), mtype='I')
    return heap

In [4]:
def test_space(spaces, remove_bad_topologies=True):
    pp = pprint.PrettyPrinter(indent=4)
    for i, space in enumerate(spaces):
        logm(f'Testing space [{i+1} of {len(spaces)}]',
             cur_frame=currentframe(), mtype='I')
        pp.pprint(space)
        try:
            K.clear_session()
            model = build_model(conf, space, input_shape=(SPEC_SHAPE_HEIGTH,
                                                          SPEC_SHAPE_WIDTH,
                                                          CHANNELS))
        except ValueError as err:
            logm(f'Failed when building the model: {str(err)} ',
                 cur_frame=currentframe(), mtype='I')
            if remove_bad_topologies:
                del space
            continue
    return spaces

In [5]:
def train(model: Model, conf: Config, batch_size, developing=False):
    logm(f'Running train for {conf}', cur_frame=currentframe(),
         mtype='I')
    train_paths, train_labels = parse_csv(conf.train_data_csv, conf.data_path)
    val_paths, val_labels = parse_csv(conf.eval_data_csv, conf.data_path)
    if developing:
        logm('Developing is set as true: limiting size of dataset',
             cur_frame=currentframe(), mtype='I')
        train_paths_labels = list(zip(train_paths, train_labels))
        random.shuffle(train_paths_labels)
        train_paths, train_labels = zip(*train_paths_labels)
        train_paths = train_paths[:100]
        train_labels = train_labels[:100]
        val_paths = val_paths[:10]
        val_labels = val_labels[:10]
        epochs = 2
        conf.steps_per_epoch = 10

    train_dataset = Dataset(train_paths, train_labels,
                            name=conf.dataset_name + '[TRAIN]',
                            num_classes=NUM_CLASSES)
    val_dataset = Dataset(val_paths, val_labels,
                          name=conf.dataset_name + '[VALIDATION]',
                          num_classes=NUM_CLASSES)
    
    epochs = int(len(train_dataset)//(batch_size*conf.steps_per_epoch)) + 1
    logm(f'Calculated number of epochs to process all data at least 1 time: {epochs}',
         cur_frame=currentframe())

    logm(f'Loading validation data...', cur_frame=currentframe(),
         mtype='I')
    X_val = np.asarray([conf.data_loader(x) for x in val_dataset()[0]])
    y_val = val_dataset()[1]
    logm(f'Loading validation data... Done', cur_frame=currentframe(),
         mtype='I')
    logm(f'Validation data shape is {X_val.shape}',
         cur_frame=currentframe(),
         mtype='I')
    if conf.use_generator:
        logm('Using generator', cur_frame=currentframe(), mtype='I')
        train_gen = Generator(paths=train_dataset()[0],
                              labels=train_dataset()[1],
                              loader_fn=conf.data_loader,
                              batch_size=int(batch_size),
                              heap=setup_heap(conf) if conf.use_heap else None,
                              expected_shape=(SPEC_SHAPE_HEIGTH,
                                              SPEC_SHAPE_WIDTH,
                                              CHANNELS))
        history = model.fit_generator(generator=train_gen,
                                      validation_data=(X_val,
                                                       y_val),
                                      use_multiprocessing=True,
                                      max_queue_size=96,
                                      workers=12,
                                      steps_per_epoch=conf.steps_per_epoch,
                                      epochs=epochs,
                                      callbacks=setup_clbks(conf),
                                      verbose=1)
    else:
        X_train = np.asarray([conf.data_loader(x) for x in train_dataset()[0]])
        y_train = train_dataset()[1]
        history = model.fit(x=X_train, y=y_train,
                            batch_size=int(batch_size),
                            validation_data=(X_val, y_val),
                            epochs=epochs,
                            callbacks=setup_clbks(conf),
                            verbose=1)
    return history, model


def random_search(conf: Config, search_space):
    logm(f'Running random search for {conf}', cur_frame=currentframe(),
         mtype='I')
    logm(f'Created report file at {conf.report_file}',
         cur_frame=currentframe(), mtype='I')

    with open(conf.report_file, 'w') as output:
        output.write(f'run,{params_keys_comma_separated()},validation_acc,'
                     'time_taken\n')

    for i, space in enumerate(search_space):
        logm(f'Running random search on space {space} - {i+1} of '
             f'{len(search_space)}', cur_frame=currentframe(), mtype='I')

        if datetime.now() > conf.time_limit:
            logm('Time limit reached: end random search',
                 cur_frame=currentframe(), mtype='I')
            return
        logm('Buiding model', cur_frame=currentframe(), mtype='I')
        try:
            K.clear_session()
            model = build_model(conf, space, input_shape=(SPEC_SHAPE_HEIGTH,
                                                          SPEC_SHAPE_WIDTH,
                                                          CHANNELS))
        except ValueError as err:
            logm(f'Error when building the model: {str(err)} ',
                 cur_frame=currentframe(), mtype='E')
            continue
        model.summary()
        with Timer() as t:
            result, model = train(model, conf, space['batch_size'],
                                  developing=DEVELOPING)
        time_taken = t.interval
    
        validation_acc = np.amax(result.history['val_acc'])
        with open(conf.report_file, 'a') as output:
            output.write(f'{conf.run},{params_values_comma_separated(space)},'
                         f'{validation_acc},{time_taken}\n')
        conf.run += 1
        with open(os.path.join(conf.log_dir, f'hist_acc_{conf.run}.csv'),
                               'w') as output:
            output.write(f'epoch,val_acc\n')
            for itr, val_acc in enumerate(result.history['val_acc']):
                output.write(f'{itr+1},{val_acc}\n')
        if SAVE_MODEL:
            model_path = os.path.join(conf.models_dir, conf.model_name +
                                      f'_{conf.run}.h5')
            logm(f'Saving model at {model_path}',
                 cur_frame=currentframe(), mtype='I')
            model.save(model_path)

In [6]:
def main(conf: Config):
    logm('>>> Started random hyperparameter search <<<')

    conf.model_name = f'random_search_{TIME_NOW}'
    logm(f'Current configuration is:\n{repr(conf)}',
         cur_frame=currentframe(), mtype='I')

    setup_dirs(conf)
    logm('Creating space', cur_frame=currentframe())
    space = create_space(RUNS)
    space = test_space(space, remove_bad_topologies=True)
    logm(f'Start random search: {str(conf)}', cur_frame=currentframe())
    with Timer() as t:
        random_search(conf, space)

    logm(f'End random search: total time taken: {str(t.interval)}',
         cur_frame=currentframe(), mtype='I')
    
    logm('End evaluation (best model): total time taken: '
            f'{str(t.interval)}', cur_frame=currentframe(), mtype='I')

In [ ]:
time_limit = datetime.strptime(TIME_LIMIT, "%d/%m/%Y %H:%M:%S")
with open(LOG_FILE, 'a') as log_file:
        log_file.write(f'\n{"="*80}\n')
        log_file.write(f'LOG SearchHyperparameters.ipynb:{datetime.now()}\n')
conf = Config(params=params,
              conf_name=f'SearchHyperparameters.ipynb_{SAMPLING_RATE}_'
              f'{MAX_SECONDS_PER_RUN}_{STEPS_PER_EPOCH}',
              data_loader=wav_to_specdata,
              use_tb_embeddings=TB_EMBEDDINGS)
conf.max_seconds_per_run = MAX_SECONDS_PER_RUN
conf.time_limit = time_limit
try:
    main(conf)
except Exception as err:
    logm(f'FATAL ERROR: {str(err)}', cur_frame=currentframe(),
         mtype='E')
    conf.delete_file()
    raise err

W0910 09:43:43.897572 140355300222784 deprecation_wrapper.py:119] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0910 09:43:43.898189 140355300222784 deprecation_wrapper.py:119] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0910 09:43:43.910832 140355300222784 deprecation_wrapper.py:119] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0910 09:43:43.911473 140355300222784 deprecation_wrapper.py:119] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placehold

2019-09-10 09:43:43.885873: I : >>> Started random hyperparameter search <<<
2019-09-10 09:43:43.886101: I <ipython-input-6-3dc90589ebd8>:6: Current configuration is:
Attributes of [SearchHyperparameters.ipynb_8000_172800_1000.d-SLD.10-9-9-43-43] configuration:
	name=SearchHyperparameters.ipynb_8000_172800_1000
	data_loader=<function wav_to_specdata at 0x7fa67ed57290>
	model_location=/home/ec2-user/lid_hyper_search/models/random_search_2019-09-10.09-43-43.375295
	batch_size=-1
	use_generator=True
	use_heap=False
	use_tb_embeddings=False
	log_dir=/home/ec2-user/lid_hyper_search/logs
	learning_rate=0.0001
	num_classes=3
	models_dir=/home/ec2-user/lid_hyper_search/models
	models_checkpoint_dir=/home/ec2-user/lid_hyper_search/models/checkpoints
	data_path=/home/ec2-user/datasets/SLD
	steps_per_epoch=1000
	epochs=8
	train_data_csv=/home/ec2-user/datasets/SLD/train.csv
	eval_data_csv=/home/ec2-user/datasets/SLD/validation.csv
	test_data_csv=None
	when=2019-09-10 09:43:43.885484
Parameters:
{

W0910 09:43:44.263319 140355300222784 deprecation_wrapper.py:119] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



2019-09-10 09:43:44.282161: I <ipython-input-4-f72fbb6363d4>:5: Testing space [2 of 100]
{   'batch_size': 8,
    'dense_1': 514,
    'dense_2': None,
    'dense_3': 256,
    'dropout1': 0.2,
    'dropout2': 0.2,
    'dropout3': 0.2,
    'dropout4': 0.4,
    'dropout5': 0.4,
    'dropout6': 0.5,
    'dropout7': 0.2,
    'dropout8': 0.4,
    'dropout_dense': 0.4,
    'filters_conv2d_1': 503,
    'filters_conv2d_2': 180,
    'filters_conv2d_3': 108,
    'filters_conv2d_4': 381,
    'filters_conv2d_5': 406,
    'filters_conv2d_6': 513,
    'filters_conv2d_7': 108,
    'filters_conv2d_8': 83,
    'lr': 0.0001,
    'maxpooling_1': None,
    'maxpooling_2': 2,
    'maxpooling_3': None,
    'maxpooling_4': None,
    'maxpooling_5': None,
    'maxpooling_6': 2,
    'maxpooling_7': None,
    'maxpooling_8': None,
    'opt': <class 'keras.optimizers.RMSprop'>}
2019-09-10 09:43:44.553159: I <ipython-input-4-f72fbb6363d4>:5: Testing space [3 of 100]
{   'batch_size': 32,
    'dense_1': 128,
    'd

2019-09-10 09:43:47.231129: I <ipython-input-4-f72fbb6363d4>:5: Testing space [12 of 100]
{   'batch_size': 16,
    'dense_1': 241,
    'dense_2': 527,
    'dense_3': None,
    'dropout1': 0.2,
    'dropout2': 0.4,
    'dropout3': 0.4,
    'dropout4': 0.2,
    'dropout5': 0.3,
    'dropout6': 0.2,
    'dropout7': 0.4,
    'dropout8': 0.3,
    'dropout_dense': 0.3,
    'filters_conv2d_1': 381,
    'filters_conv2d_2': 30,
    'filters_conv2d_3': 85,
    'filters_conv2d_4': 115,
    'filters_conv2d_5': 13,
    'filters_conv2d_6': 98,
    'filters_conv2d_7': 53,
    'filters_conv2d_8': 47,
    'lr': 1e-05,
    'maxpooling_1': None,
    'maxpooling_2': 2,
    'maxpooling_3': 2,
    'maxpooling_4': None,
    'maxpooling_5': 2,
    'maxpooling_6': None,
    'maxpooling_7': 2,
    'maxpooling_8': 2,
    'opt': <class 'keras.optimizers.Adagrad'>}
2019-09-10 09:43:47.621345: I <ipython-input-4-f72fbb6363d4>:5: Testing space [13 of 100]
{   'batch_size': 8,
    'dense_1': 492,
    'dense_2': 114,

...

2019-09-10 09:44:07.963912: I <ipython-input-4-f72fbb6363d4>:5: Testing space [82 of 100]
{   'batch_size': 8,
    'dense_1': 251,
    'dense_2': 512,
    'dense_3': 256,
    'dropout1': 0.4,
    'dropout2': 0.3,
    'dropout3': 0.4,
    'dropout4': 0.3,
    'dropout5': 0.4,
    'dropout6': 0.2,
    'dropout7': 0.2,
    'dropout8': 0.4,
    'dropout_dense': 0.4,
    'filters_conv2d_1': 380,
    'filters_conv2d_2': 46,
    'filters_conv2d_3': 187,
    'filters_conv2d_4': 187,
    'filters_conv2d_5': 286,
    'filters_conv2d_6': 30,
    'filters_conv2d_7': 205,
    'filters_conv2d_8': 12,
    'lr': 0.001,
    'maxpooling_1': 2,
    'maxpooling_2': None,
    'maxpooling_3': None,
    'maxpooling_4': None,
    'maxpooling_5': 2,
    'maxpooling_6': None,
    'maxpooling_7': None,
    'maxpooling_8': 2,
    'opt': <class 'keras.optimizers.RMSprop'>}
2019-09-10 09:44:08.256682: I <ipython-input-4-f72fbb6363d4>:5: Testing space [83 of 100]
{   'batch_size': 32,
    'dense_1': 513,
    'dense_

2019-09-10 09:44:10.961642: I <ipython-input-4-f72fbb6363d4>:5: Testing space [92 of 100]
{   'batch_size': 32,
    'dense_1': 506,
    'dense_2': 264,
    'dense_3': 256,
    'dropout1': 0.2,
    'dropout2': 0.2,
    'dropout3': 0.2,
    'dropout4': 0.5,
    'dropout5': 0.5,
    'dropout6': 0.5,
    'dropout7': 0.4,
    'dropout8': 0.3,
    'dropout_dense': 0.4,
    'filters_conv2d_1': 113,
    'filters_conv2d_2': None,
    'filters_conv2d_3': 20,
    'filters_conv2d_4': 81,
    'filters_conv2d_5': 316,
    'filters_conv2d_6': 79,
    'filters_conv2d_7': None,
    'filters_conv2d_8': 280,
    'lr': 1e-06,
    'maxpooling_1': 2,
    'maxpooling_2': None,
    'maxpooling_3': 2,
    'maxpooling_4': 2,
    'maxpooling_5': 2,
    'maxpooling_6': 2,
    'maxpooling_7': 2,
    'maxpooling_8': 2,
    'opt': <class 'keras.optimizers.Adam'>}
2019-09-10 09:44:11.342608: I <ipython-input-4-f72fbb6363d4>:5: Testing space [93 of 100]
{   'batch_size': 16,
    'dense_1': None,
    'dense_2': 264,
  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 81, 499, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 81, 499, 392)      3920      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 250, 392)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 41, 250, 392)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 39, 248, 62)       218798    
_________________________________________________________________
dropout_2 (Dropout)          (None, 39, 248, 62)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 246, 311)      173849    
__________

W0910 09:44:30.229585 140355300222784 deprecation.py:323] From /home/ec2-user/deeplearning/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-09-10 09:44:30.092436: I <ipython-input-3-f40387e03c4c>:40: Setting up callbacks
2019-09-10 09:44:30.092686: I <ipython-input-3-f40387e03c4c>:83: Setting up TimedStopping
2019-09-10 09:44:30.093068: I <ipython-input-3-f40387e03c4c>:107: Setting up callbacks... Done
Epoch 1/85
1000/1000 [==============================] - 2368s 2s/step - loss: 1.0837 - acc: 0.3693 - val_loss: 1.0994 - val_acc: 0.2679
2019-09-10 10:23:59.030670: I /home/ec2-user/lid_hyper_search/common/callbacks.py:50: Average elapsed time: 170432.16003966331
Epoch 2/85
1000/1000 [==============================] - 2371s 2s/step - loss: 1.0715 - acc: 0.3838 - val_loss: 1.0888 - val_acc: 0.2679
2019-09-10 11:03:29.642289: I /home/ec2-user/lid_hyper_search/common/callbacks.py:50: Average elapsed time: 169246.85421288013
Epoch 3/85
1000/1000 [==============================] - 2377s 2s/step - loss: 1.0502 - acc: 0.4228 - val_loss: 1.0530 - val_acc: 0.3832
2019-09-10 11:43:06.233081: I /home/ec2-user/lid_hyper_search/commo